# Łódź University of Technology
## Faculty of Electrotechnics, Electronics, Information Technology and Automation Technology
## Institution of Medical Electronics

### Created by Dominik Dmowski
___

## Import Libraries

In [6]:
import os
import cv2
import matplotlib.pyplot as plt
import albumentations as A
import numpy as np
import random
import time

## Functions for output visualizations

I mainly used those functions for testing, in this version there is no use of them, so you can feel free to skip that step.

In [7]:
# The first function is from the official Albumentations documentations
# It has been used to visualize sigular images during the augmentation process
def visualize(image):
    
    """For a single image visualization, 
    the input parameter is an image that will be displayed"""

    plt.figure(figsize=(10, 10))
    plt.axis('off')
    plt.imshow(image)

# The second function is created by my own, it is extended version of previous one
# The idea was to visualize both image and mask in one row
def multi_visualize(image, mask):
    
    """For a multiple visualization - as a result there will be displayed a pair of images in one line. 
    As input there are two singular images, the primary usage was to display a pair of an image and its mask."""
    
    fig = plt.figure(figsize=(10, 10))
    fig.add_subplot(2,2,1)
    plt.axis('off')
    plt.imshow(image)
    fig.add_subplot(2,2,2)
    plt.axis('off')
    plt.imshow(mask)

## Augmentation function

In [13]:
def image_augmentation(image_path):
    
    """A function that allow to read an image by its path and
    create an augmentation of that variable"""
    
    # read image
    image = cv2.imread(image_path)
    
    # create transformation
    transform = A.Compose([
        A.GaussianBlur (blur_limit=(3, 3), sigma_limit=0, p=.5),
        A.RandomGamma (gamma_limit=(80, 120), eps=None, p=.5),
        A.RandomBrightnessContrast (brightness_limit=0.2, contrast_limit=0.2, 
                                    brightness_by_max=True, p=.5),
        A.HorizontalFlip (p=.5),
        A.VerticalFlip (p=.5),
        A.SafeRotate (limit=90, interpolation=1, border_mode=cv2.BORDER_CONSTANT, 
                      value=None, mask_value=None, p=.5),
        A.Perspective (scale=(0.05, 0.1), keep_size=True, pad_mode=cv2.BORDER_CONSTANT, 
                       pad_val=0, mask_pad_val=0, fit_output=False, interpolation=1, p=.5),
        A.ElasticTransform (alpha=1, sigma=15, alpha_affine=10, interpolation=1, 
                            border_mode=cv2.BORDER_CONSTANT, value=None, mask_value=None, p=.5)
    ])
    
    # apply augmentation
    augmented = transform(image=image)
    
    # retrive image
    augmented_image = augmented['image']
    
    # return variable
    return augmented_image

## Creat filename function

In [9]:
def create_filename(orginal_filename, iterator):
    
    """Function to create new filenames in pipeline for augmented images.
     As an input it takes orginal filename and the loop iterator"""
    
    
    # Find and divide name and extension
    split_filename = orginal_filename.split('.')
    
    # Add to name the iterator
    split_filename[0] = split_filename[0] + "_aug_" + str(iterator)
    
    # merge the filename with extension
    created_name = ".".join(split_filename)
    
    # return merged value
    return created_name

## Folder paths that are storing the information

In [10]:
# Little structure explanation

# The Kidney Images are divided into 3 classes:
#      - Control - kidneys of those patients were classified as healthy
#      - Active inflamation (AI) - in those kidney images the doctor noticed inflammatory changes
#      - Chronic fibrosis (CF) - in those images the doctor noticed the fibroid structures

main_directory = r'Dataset-kidney-MRI\Training'
control_directory = r'Dataset-kidney-MRI\Training\0'
ai_directory = r'Dataset-kidney-MRI\Training\1'
cf_directory = r'Dataset-kidney-MRI\Training\2'

# The augmentation will be done only on the training data
# The test data will be unchanged

## Loop to create image augmentations

In [14]:
# Get previously defined directories paths 
# into table to easily iterate them in a loop
directories = [control_directory, 
              ai_directory,
              cf_directory]

# Set file counter and starting time
# to calculate number of generate images and duration of execution
file_counter = 0
time_start = time.time()



#iterate by every directory
for i in range(0,3):
    
    # Get the correct folder directory for each iteration
    current_directory = directories[i]
    
    
    # iterate by every file in current directory
    for filename in os.listdir(current_directory):
        
        # create whole file path
        file_path = os.path.join(current_directory,
                                 filename)
        
        # Loop for the augmenation operations
        for i in range(0,20):
            
            # augment an image
            new_image = image_augmentation(file_path)
            
            # create file name and path
            new_filename = create_filename(filename, i)
            new_filepath = os.path.join(current_directory,
                                        new_filename)
            
            # save the file
            cv2.imwrite(new_filepath, new_image)
            
            # Acumulate number of files
            file_counter += 1
        
        
# Get the finishing time and count the duration        
time_finish = time.time()
duration = time_finish - time_start

print("Process has been done in " + str(round(duration, 3)) +
      " and generated " + str(file_counter) + " images.")

Process has been done in 68.327 and generated 20400 images.
